# АДЭФС. Лабораторная работа №1

**Формат:** Jupyter notebook

Краткое содержание: расчет описательных статистик, визуализация распределений, корреляционный анализ.


## Описание набора данных

Файл: `/mnt/data/99de4342-b513-4963-9a3a-0f98df3f6a3e.csv`. Ниже — первые 10 строк и типы столбцов.

In [ ]:
import pandas as pd

df = pd.read_csv(r'/mnt/data/99de4342-b513-4963-9a3a-0f98df3f6a3e.csv')
df.head(10)

df.dtypes

## 1. Расчет описательных статистик

Для каждого числового столбца вычислены: минимум, максимум, размах, среднее, дисперсия, стандартное отклонение, медиана и квантили 0.01, 0.05, 0.95, 0.99.

**Код и результаты ниже.**

In [ ]:
import numpy as np
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
res = pd.DataFrame(index=num_cols, columns=['min','max','range','mean','var','std','median','q0.01','q0.05','q0.95','q0.99'])
for c in num_cols:
    s = df[c].dropna()
    res.loc[c,'min'] = s.min()
    res.loc[c,'max'] = s.max()
    res.loc[c,'range'] = s.max() - s.min()
    res.loc[c,'mean'] = s.mean()
    res.loc[c,'var'] = s.var(ddof=1)
    res.loc[c,'std'] = s.std(ddof=1)
    res.loc[c,'median'] = s.median()
    res.loc[c,'q0.01'] = s.quantile(0.01)
    res.loc[c,'q0.05'] = s.quantile(0.05)
    res.loc[c,'q0.95'] = s.quantile(0.95)
    res.loc[c,'q0.99'] = s.quantile(0.99)
res = res.astype(float)
res.round(6)


## 2. Визуализация распределений

Гистограммы для каждого числового столбца (один график на фигуру). Также вычисляется скошенность и эксцесс для кратких выводов.

In [ ]:
import matplotlib.pyplot as plt
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
for c in num_cols:
    s = df[c].dropna()
    plt.figure(figsize=(6,4))
    plt.hist(s, bins='auto')
    plt.title(f'Гистограмма: {c}')
    plt.xlabel(c)
    plt.ylabel('Частота')
    plt.tight_layout()
    plt.show()
    skew = s.skew()
    kurt = s.kurtosis()
    print(f'Столбец: {c} — скошенность (skew) = {skew:.4f}, эксцесс (kurtosis) = {kurt:.4f}')


## 3. Корреляционный анализ

Вычисляем матрицу корреляций Пирсона и соответствующие p-values. Для пар с |r| >= 0.7 строим scatter plot.

In [ ]:
from scipy import stats
import pandas as pd
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
corr = df[num_cols].corr(method='pearson')
pvals = pd.DataFrame(index=num_cols, columns=num_cols, dtype=float)
for i in range(len(num_cols)):
    for j in range(len(num_cols)):
        if i == j:
            pvals.iloc[i,j] = 0.0
        else:
            s1 = df[num_cols[i]]
            s2 = df[num_cols[j]]
            common = s1.dropna().index.intersection(s2.dropna().index)
            if len(common) < 2:
                pvals.iloc[i,j] = np.nan
            else:
                r, p = stats.pearsonr(s1.loc[common], s2.loc[common])
                pvals.iloc[i,j] = p

print('Матрица корреляций:')
display(corr.round(4))
print('\nМатрица p-values:')
display(pvals.round(6))

# strong pairs
strong = []
for i in range(len(num_cols)):
    for j in range(i+1, len(num_cols)):
        r = corr.iloc[i,j]
        p = pvals.iloc[i,j]
        if pd.notna(r) and abs(r) >= 0.7:
            strong.append((num_cols[i], num_cols[j], r, p))

print('\nСильно коррелирующие пары (|r| >= 0.7):')
if len(strong) == 0:
    print('Нет сильно коррелирующих пар с |r| >= 0.7')
else:
    for a,b,r,p in strong:
        print(f'{a} — {b}: r = {r:.4f}, p = {p:.4g}')

for a,b,r,p in strong:
    plt.figure(figsize=(5,4))
    plt.scatter(df[a], df[b])
    plt.title(f'Scatter: {a} vs {b} — r={r:.3f}, p={p:.3g}')
    plt.xlabel(a)
    plt.ylabel(b)
    plt.tight_layout()
    plt.show()


## Общие выводы

Опишите кратко основные наблюдения: распределения, наличие/отсутствие сильных корреляций, качество данных (пропуски/выбросы) и рекомендации по дальнейшему анализу.